In [51]:
import numpy as np
import pandas as pd
asyncs = ["no interaction","unassisted breath","early trigger","late trigger",
          "failed trigger","false trigger","multiple trigger","early cycling",
          "late cycling", "ambiguous", "expiratory work", "artefact", "work shifting"]
dev = pd.read_csv("../data/train/combined.csv").rename({"splinting" : "ambiguous", "no asynchrony" : "no interaction"}, axis=1)
test = pd.read_csv("../data/test/combined.csv").rename({"splinting" : "ambiguous", "no asynchrony" : "no interaction"}, axis=1)

In [52]:
# Supplementary Table 2
dev_label_counts = dev.groupby("Sample")[asyncs].sum()
dev_label_counts["Data Partition"] = "Development"
test_label_counts = test.groupby("Sample")[asyncs].sum()
test_label_counts["Data Partition"] = "Test"
label_counts = pd.concat((dev_label_counts, test_label_counts))
label_counts["Mode"] = "AC"
label_counts.loc[["DG204","DG212","DG220"], "Mode"] = "SIMV"
label_counts.loc["DG222", "Mode"] = "PSV"
label_counts

,no interaction,unassisted breath,early trigger,late trigger,failed trigger,false trigger,multiple trigger,early cycling,late cycling,ambiguous,expiratory work,artefact,work shifting,Data Partition,Mode
Sample,,,,,,,,,,,,,,,
DG201,353,13,13,20,36,3,0,3,12,3,66,6,0,Development,AC
DG202,239,30,0,8,4,0,6,5,38,3,200,9,0,Development,AC
DG203,154,19,9,32,31,1,3,13,51,5,253,23,0,Development,AC
DG204,18,349,1,1,27,0,0,12,1,9,125,1,17,Development,SIMV
DG205,118,1,19,10,27,0,3,1,286,17,123,11,0,Development,AC
DG206,372,0,6,8,6,2,0,2,44,1,76,5,0,Development,AC
DG207,311,21,5,23,4,0,3,3,53,2,107,10,0,Development,AC
DG210,181,63,9,20,23,0,9,7,40,12,181,30,0,Development,AC
DG212,108,131,2,22,40,0,2,8,7,4,201,36,2,Development,SIMV


In [53]:
# Supplementary Table 3
vb_counts = 500 - label_counts["unassisted breath"]
async_counts = vb_counts - label_counts[["no interaction", "artefact"]].sum(axis=1)
ai = pd.DataFrame({"Asynchrony Index" : async_counts / vb_counts, 
                   "Number of Breaths" : vb_counts,
                   "Mode" : label_counts["Mode"]}).reset_index()
ai.sort_values(["Mode", "Sample"])

,Sample,Asynchrony Index,Number of Breaths,Mode
0,DG201,0.262834,487,AC
1,DG202,0.472340,470,AC
2,DG203,0.632017,481,AC
4,DG205,0.741483,499,AC
5,DG206,0.246000,500,AC
6,DG207,0.329854,479,AC
7,DG210,0.517162,437,AC
9,DG213,0.554598,348,AC
10,DG214,0.551198,459,AC
11,DG215,0.672165,485,AC


In [54]:
ai.groupby("Mode")["Asynchrony Index"].mean()

Mode
AC      0.502015
PSV     0.419878
SIMV    0.708107
Name: Asynchrony Index, dtype: float64

In [55]:
# Table 3 Overall
pd.DataFrame({"Percentage" : (label_counts[asyncs].sum(axis=0) / 11500),
              "Count" : label_counts[asyncs].mean(axis=0)}).sort_values("Percentage", ascending=False)

,Percentage,Count
no interaction,0.402435,201.217391
expiratory work,0.311826,155.913043
unassisted breath,0.122696,61.347826
late cycling,0.099304,49.652174
late trigger,0.059391,29.695652
failed trigger,0.042696,21.347826
artefact,0.031913,15.956522
ambiguous,0.016087,8.043478
multiple trigger,0.013130,6.565217
early trigger,0.012261,6.130435


In [56]:
# Table 3 distribution stats overall
pd.DataFrame({"25th Percentile Percentage" : (label_counts[asyncs].quantile(q=0.25, axis=0, interpolation="nearest") / 500),
              "25th Percentile Count" : label_counts[asyncs].quantile(q=0.25, axis=0, interpolation="nearest"),
              "Median Percentage" : (label_counts[asyncs].quantile(axis=0, interpolation="nearest") / 500),
              "Median Count" : label_counts[asyncs].quantile(axis=0, interpolation="nearest"),
              "75th Percentile Percentage" : (label_counts[asyncs].quantile(q=0.75, axis=0, interpolation="nearest") / 500),
              "75th Percentile Count" : label_counts[asyncs].quantile(q=0.75, axis=0, interpolation="nearest")}).sort_values("Median Percentage", ascending=False)

,25th Percentile Percentage,25th Percentile Count,Median Percentage,Median Count,75th Percentile Percentage,75th Percentile Count
no interaction,0.274,137,0.398,199,0.554,277
expiratory work,0.232,116,0.284,142,0.402,201
late cycling,0.028,14,0.076,38,0.102,51
failed trigger,0.012,6,0.046,23,0.062,31
late trigger,0.028,14,0.044,22,0.074,37
unassisted breath,0.004,2,0.030,15,0.082,41
artefact,0.016,8,0.028,14,0.046,23
ambiguous,0.006,3,0.014,7,0.020,10
early trigger,0.004,2,0.008,4,0.018,9
multiple trigger,0.000,0,0.008,4,0.016,8


In [57]:
# Table 3 Development
pd.DataFrame({"Percentage" : (label_counts.loc[label_counts["Data Partition"] == "Development"][asyncs].sum(axis=0) / 9000),
              "Count" : label_counts.loc[label_counts["Data Partition"] == "Development"][asyncs].mean(axis=0)}).sort_values("Percentage", ascending=False)

,Percentage,Count
no interaction,0.384556,192.277778
expiratory work,0.296222,148.111111
unassisted breath,0.153111,76.555556
late cycling,0.106222,53.111111
late trigger,0.062556,31.277778
failed trigger,0.040333,20.166667
artefact,0.032111,16.055556
ambiguous,0.015444,7.722222
early cycling,0.010667,5.333333
early trigger,0.009778,4.888889


In [58]:
# Table 3 Test
pd.DataFrame({"Percentage" : (label_counts.loc[label_counts["Data Partition"] == "Test"][asyncs].sum(axis=0) / 2500),
              "Count" : label_counts.loc[label_counts["Data Partition"] == "Test"][asyncs].mean(axis=0)}).sort_values("Percentage", ascending=False)

,Percentage,Count
no interaction,0.4668,233.4
expiratory work,0.3680,184.0
late cycling,0.0744,37.2
failed trigger,0.0512,25.6
late trigger,0.0480,24.0
artefact,0.0312,15.6
multiple trigger,0.0296,14.8
early trigger,0.0212,10.6
ambiguous,0.0184,9.2
unassisted breath,0.0132,6.6


In [59]:
combined = pd.concat((dev, test))
asyncs = np.array(["early trigger","late trigger", "failed trigger",
          "multiple trigger","early cycling", "late cycling", "expiratory work"])
asyncs_df = combined.loc[(combined["unassisted breath"] == 0) & 
             (combined["no interaction"] == 0)]

In [60]:
asyncs_df[asyncs].sum(axis=1).value_counts()

1    4222
2     851
0     271
3     105
4      12
6       1
Name: count, dtype: int64

In [61]:
# Table 4
from scipy.stats import fisher_exact
pvi_pairs = []
for i in range(len(asyncs)-1):
    for j in range(i+1, len(asyncs)):
        a = asyncs[i]
        b = asyncs[j]
        mat = np.zeros((2,2))
        mat[0,0] = sum((combined[a] == 0) & (combined[b] == 0))
        mat[0,1] = sum((combined[a] == 0) & (combined[b] == 1))
        mat[1,0] = sum((combined[a] == 1) & (combined[b] == 0))
        mat[1,1] = sum((combined[a] == 1) & (combined[b] == 1))
        fish = fisher_exact(mat)
        pvi_pairs.append({"PVI Pair" : "{}-{}".format(a,b), "Odd's Ratio" : fish[0], "p-value" : fish[1]})
pd.DataFrame(pvi_pairs).sort_values("p-value")

,PVI Pair,Odd's Ratio,p-value
16,multiple trigger-late cycling,9.000024,6.304235e-34
14,failed trigger-expiratory work,2.622457,6.735500e-25
1,early trigger-failed trigger,6.971678,3.654605e-15
9,late trigger-late cycling,2.229657,4.636400e-13
15,multiple trigger-early cycling,13.606652,1.497556e-12
20,late cycling-expiratory work,0.657550,3.576275e-09
19,early cycling-expiratory work,2.148755,6.266874e-05
17,multiple trigger-expiratory work,1.823695,3.683944e-04
0,early trigger-late trigger,0.000000,4.118949e-04
13,failed trigger-late cycling,0.537165,8.656578e-04
